In [1]:
# Load voice files from drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Import statements
import librosa
import numpy as np
import pandas as pd
import scipy
import csv
import sys
import glob
import pickle

In [ ]:
# Load voice files using Librosa
def load_mp3(filename):
    audio_path = '/content/drive/MyDrive/Colab Notebooks/' + filename
    y , sr = librosa.load(audio_path)
    return y , sr

In [ ]:
# Statistical analysis of voice file frequency
def describe_freq(freqs):
    q1 = np.quantile(freqs, 0.25)
    iqr = scipy.stats.iqr(freqs)
    
    return [abs(q1), abs(iqr)]


In [ ]:
# Load CSV metadata
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cv-valid-train.csv')
data.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-train/sample-000000.mp3,learn to recognize omens and follow them the o...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-train/sample-000001.mp3,everything in the universe evolved he said,1,0,NaN,NaN,NaN,NaN
2,cv-valid-train/sample-000002.mp3,you came so that you could learn about your dr...,1,0,NaN,NaN,NaN,NaN
3,cv-valid-train/sample-000003.mp3,so now i fear nothing because it was those ome...,1,0,NaN,NaN,NaN,NaN
4,cv-valid-train/sample-000004.mp3,if you start your emails with greetings let me...,3,2,NaN,NaN,NaN,NaN


In [ ]:
data.count()

filename      73466
text          73466
up_votes      73466
down_votes    73466
age           73466
gender        73466
accent        63163
dtype: int64

In [ ]:
# Remove na columns and irrelavant columns
data = data.drop(columns=['duration'])
data = data.dropna(subset=['age','gender'])

In [ ]:
  import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Narrow dataset for partial feature extraction
data.reset_index(drop=True, inplace=True)
EDA_data = data[6000:7000]
EDA_data

,filename,text,up_votes,down_votes,age,gender,accent
6000,cv-valid-train/sample-016140.mp3,it didn't bring with it the smell of the deser...,3,0,fifties,male,us
6001,cv-valid-train/sample-016142.mp3,let me tell you what will happen,3,0,teens,female,england
6002,cv-valid-train/sample-016145.mp3,they locked themselves in their laboratories a...,3,0,fourties,male,us
6003,cv-valid-train/sample-016147.mp3,this samantha tommy movement costs you a fortune,1,0,thirties,male,england
6004,cv-valid-train/sample-016149.mp3,you must always know what it is that you want ...,3,0,thirties,male,england
...,...,...,...,...,...,...,...
6995,cv-valid-train/sample-018880.mp3,i've had a dozen calls already,1,0,thirties,male,us
6996,cv-valid-train/sample-018881.mp3,my review of the sun one star,3,0,thirties,male,scotland
6997,cv-valid-train/sample-018883.mp3,handle that with kid gloves,1,0,teens,female,african
6998,cv-valid-train/sample-018884.mp3,i'm just your little rent collector,1,0,fifties,female,australia


In [ ]:
# Extract features into dataframe
# Features extracted : Spectral Centroid, Root Mean Squared Energy, MFCC, Fundamental Frequency, Interquartile Range, Q1, Chroma, Mel Spectrogram
spectral_centroid_list = []
rmse_list = []
mfcc_list = []
ff_list = []
iqr = []
q1 = []
chroma_list = []
mel_list = []

count = 1
for filename in EDA_data['filename']:
    print("Processing file " + str(count))
    mp3 = load_mp3(filename)

    freq = np.fft.fft(mp3[0])
    stats = describe_freq(freq)

    spectral_centroid=librosa.feature.spectral_centroid(mp3[0])[0]
    
    rmse = librosa.feature.rms(mp3[0])[0]
    rmse_list.append(rmse.mean())

    mfcc = librosa.feature.mfcc(mp3[0])

    ff = librosa.pyin(mp3[0], librosa.note_to_hz('C2'), librosa.note_to_hz('C7'))[0]
    ff = np.nanmean(ff)

    stft = np.abs(librosa.stft(mp3[0]))
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=mp3[1]).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(mp3[0], sr=mp3[1]).T,axis=0)

    spectral_centroid_list.append(spectral_centroid.mean())
    mfcc_list.append(mfcc)
    ff_list.append(ff)
    q1.append(stats[0])
    iqr.append(stats[1])
    chroma_list.append(chroma)
    mel_list.append(mel)

    count += 1


EDA_data['mean_RMSE'] = rmse_list
EDA_data['mean_spec_centroid'] = spectral_centroid_list
EDA_data['mfcc'] = mfcc_list
EDA_data['meanff'] = ff_list
EDA_data['q1'] = q1
EDA_data['iqr'] = iqr
EDA_data['chroma'] = chroma_list
EDA_data['mel'] = mel_list


Processing file 1
Processing file 2
Processing file 3
Processing file 4
Processing file 5
Processing file 6
Processing file 7
Processing file 8
Processing file 9
Processing file 10
Processing file 11
Processing file 12
Processing file 13
Processing file 14
Processing file 15
Processing file 16
Processing file 17
Processing file 18
Processing file 19
Processing file 20
Processing file 21
Processing file 22
Processing file 23
Processing file 24
Processing file 25
Processing file 26
Processing file 27
Processing file 28
Processing file 29
Processing file 30
Processing file 31
Processing file 32
Processing file 33
Processing file 34
Processing file 35
Processing file 36
Processing file 37
Processing file 38
Processing file 39
Processing file 40
Processing file 41
Processing file 42
Processing file 43
Processing file 44
Processing file 45
Processing file 46
Processing file 47
Processing file 48
Processing file 49
Processing file 50
Processing file 51
Processing file 52
Processing file 53
Pr

In [4]:
# Convert dataframe to pickle for loading later

# EDA_data.to_pickle('6000-7000_mfcc_EDA_data')
#df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ML Pickle Jar/PYTHON3.6_15000-16000_mfcc_EDA_data")

# Load pickle files from folder
df_list = []
for filename in glob.glob("/content/drive/MyDrive/Colab Notebooks/Code for submission/ML Pickle Jar/*"):
  print("Processing " + filename)
  df = pd.read_pickle(filename)
  df_list.append(df)

sound_data = pd.concat(df_list, axis=0)
sound_data.count()

Processing /content/drive/MyDrive/Colab Notebooks/Code for submission/ML Pickle Jar/1000-2000_mfcc_EDA_data
Processing /content/drive/MyDrive/Colab Notebooks/Code for submission/ML Pickle Jar/2000-3000_mfcc_EDA_data
Processing /content/drive/MyDrive/Colab Notebooks/Code for submission/ML Pickle Jar/3000-4000_mfcc_EDA_data
Processing /content/drive/MyDrive/Colab Notebooks/Code for submission/ML Pickle Jar/4000-5000_mfcc_EDA_data
Processing /content/drive/MyDrive/Colab Notebooks/Code for submission/ML Pickle Jar/5000-6000_mfcc_EDA_data
Processing /content/drive/MyDrive/Colab Notebooks/Code for submission/ML Pickle Jar/6000-7000_mfcc_EDA_data
Processing /content/drive/MyDrive/Colab Notebooks/Code for submission/ML Pickle Jar/0001-1000_mfcc_EDA_data
Processing /content/drive/MyDrive/Colab Notebooks/Code for submission/ML Pickle Jar/7000-8000_mfcc_EDA_data
Processing /content/drive/MyDrive/Colab Notebooks/Code for submission/ML Pickle Jar/8000-9000_mfcc_EDA_data
Processing /content/drive/My

filename              73465
text                  73465
up_votes              73465
down_votes            73465
age                   73465
gender                73465
accent                63163
mean_RMSE             73465
mean_spec_centroid    73465
mfcc                  73465
meanff                72999
q1                    73465
iqr                   73465
chroma                73465
mel                   73465
dtype: int64

In [5]:
# New dataframe with features
sound_data = sound_data.sort_index(axis=0) 
sound_data

,filename,text,up_votes,down_votes,age,gender,accent,mean_RMSE,mean_spec_centroid,mfcc,meanff,q1,iqr,chroma,mel
0,cv-valid-train/sample-000005.mp3,a shepherd may like to travel but he should ne...,1,0,twenties,female,us,0.006932,1872.068718,"[[-752.60504, -730.3535, -712.5404, -711.55, -...",193.083434,0.211150,0.398957,"[0.50963885, 0.4695945, 0.4603954, 0.48003745,...","[0.00023007525, 0.00014125524, 0.00015047414, ..."
1,cv-valid-train/sample-000008.mp3,put jackie right on the staff,3,0,seventies,male,us,0.015068,2392.555799,"[[-682.6077, -657.9619, -592.8748, -546.271, -...",73.776776,0.350835,0.472287,"[0.55314964, 0.5806079, 0.69629747, 0.72144514...","[0.00015507598, 0.00031110694, 0.0020643708, 0..."
2,cv-valid-train/sample-000013.mp3,but he had found a guide and didn't want to mi...,1,0,thirties,female,us,0.038121,1863.409810,"[[-594.4625, -478.59122, -434.48297, -434.5027...",197.166506,0.477443,3.371958,"[0.57519054, 0.6260021, 0.56402045, 0.50898755...","[0.00035774417, 0.0002847947, 0.00039127853, 0..."
3,cv-valid-train/sample-000014.mp3,as they began to decorate the hallway a silhou...,1,0,sixties,male,england,0.032334,1836.372026,"[[-599.1911, -470.68686, -428.93854, -469.1475...",152.810185,0.791062,1.634413,"[0.5638525, 0.5700067, 0.6283644, 0.6683084, 0...","[0.0050621736, 0.00480701, 0.0048234495, 0.060..."
4,cv-valid-train/sample-000019.mp3,then they got ahold of some dough and went goofy,1,0,fifties,male,australia,0.045087,1870.815465,"[[-661.824, -661.824, -535.77356, -303.3603, -...",96.892631,1.543091,3.045843,"[0.6743262, 0.67634916, 0.71708095, 0.755776, ...","[1.1510695, 2.558978, 6.1211457, 9.659585, 7.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73461,cv-valid-train/sample-195766.mp3,but before i go i want to tell you a little story,1,0,fourties,male,england,0.112117,1874.815789,"[[-441.80307, -345.83603, -316.07602, -327.398...",114.783629,1.377083,2.650222,"[0.6611608, 0.5841805, 0.5251608, 0.4927581, 0...","[0.23971552, 0.34807414, 2.7860074, 27.334885,..."
73462,cv-valid-train/sample-195767.mp3,down below in the darkness were hundreds of pe...,3,0,thirties,female,us,0.007726,1956.440242,"[[-750.32196, -723.32166, -706.97534, -702.659...",175.643492,0.723267,1.596329,"[0.6474357, 0.6757682, 0.66542614, 0.65826356,...","[0.00011484503, 0.00019641899, 0.0004751418, 0..."
73463,cv-valid-train/sample-195770.mp3,he heard a muffled grating sound and saw the b...,4,0,twenties,male,england,0.042985,1575.157235,"[[-660.3753, -660.3753, -547.0556, -382.08624,...",85.790579,6.728792,4.414121,"[0.71263987, 0.72187686, 0.6750175, 0.6594301,...","[0.35012475, 0.91567373, 7.2036114, 8.911719, ..."
73464,cv-valid-train/sample-195771.mp3,the englishman said nothing,1,0,thirties,male,england,0.018556,2302.709109,"[[-700.3081, -700.3081, -700.3081, -698.8642, ...",121.380871,1.061520,2.111955,"[0.5877028, 0.5170463, 0.516343, 0.5301599, 0....","[0.0013285666, 0.0012072835, 0.003190259, 0.18..."


In [6]:
sound_data['gender'].value_counts()

male      54592
female    18099
other       774
Name: gender, dtype: int64

In [7]:
sound_data['age'].value_counts()

twenties     22926
thirties     18287
fourties     10914
fifties       9442
teens         5441
sixties       4584
seventies     1632
eighties       239
Name: age, dtype: int64